In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from rich.progress import Progress

In [2]:
# Define dataset
class MoodDataset(Dataset):
    def __init__(self, features_1, features_2, labels):
        self.features_1 = features_1
        self.features_2 = features_2
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Convert features and labels to tensors
        features_1 = torch.from_numpy(self.features_1[idx]).float()
        features_2 = torch.from_numpy(self.features_2[idx]).float()
        labels = torch.from_numpy(np.array(self.labels[idx])).float()
        return features_1, features_2, labels

In [3]:
# Define model
class TransformerModel(nn.Module):
    def __init__(self):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model=21, nhead=1)
        self.fc = nn.Linear(21, 1)

    def forward(self, src, tgt):
        x = self.transformer(src, tgt)
        x = self.fc(x)
        return x

In [4]:
# Load data
df = pd.read_csv('time_resampling/featured_time_resamping_sparse_matrix_data.csv')

# Group by id
grouped = df.groupby('id')

# Data preparation
features_1, features_2, labels = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for name, group in grouped:
    feature_1 = group.drop(['id', 'time'], axis=1)
    feature_2 = group.drop(['id', 'time'], axis=1)
    label = group['mood']

    feature_1 = feature_1.iloc[:-1].reset_index(drop=True)
    feature_2 = feature_2.iloc[1:].reset_index(drop=True)
    label = label.iloc[1:].reset_index(drop=True)
    
    features_1 = pd.concat([features_1, feature_1])
    features_2 = pd.concat([features_2, feature_2])
    labels = pd.concat([labels, label])

# Split into training and test sets
train_features_1, test_features_1, train_features_2, test_features_2, train_labels, test_labels = train_test_split(features_1.values, features_2.values, labels.values, test_size=0.2, shuffle=False)
train_features_1, test_features_1, train_features_2, test_features_2, train_labels, test_labels = train_features_1.astype(np.float32), test_features_1.astype(np.float32), train_features_2.astype(np.float32), test_features_2.astype(np.float32), train_labels.astype(np.float32), test_labels.astype(np.float32)

# Standardize features
# scaler = StandardScaler()
# train_features = scaler.fit_transform(train_features)
# test_features = scaler.transform(test_features)

# Create data loaders
train_dataset = MoodDataset(train_features_1, train_features_2, train_labels)
test_dataset = MoodDataset(test_features_1, test_features_2, test_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Initialize model, optimizer and loss function
model = TransformerModel().to(device)
optimizer = AdamW(model.parameters(), lr=0.00001)
criterion = nn.MSELoss()

/home/loong/miniconda3/envs/PyTorch/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
# Training
with Progress() as progress:
    task = progress.add_task("[cyan]Training...", total=len(50))
    for epoch in range(50):
        total_loss = 0
        total_correct = 0
        total_count = 0
        
        for i, (features_1, features_2, labels) in enumerate(train_loader):
            src = features_1.float().to(device)
            tgt = features_2.float().to(device)
            labels = labels.float().squeeze().to(device)

            optimizer.zero_grad()

            outputs = model(src, tgt).squeeze()
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # Calculate total loss and accuracy
            total_loss += loss.item()
            # outputs和labels为list，若outputs和labels中的一项的差值小于0.1，则认为这一项预测正确
            total_correct += sum([1 for i in range(len(outputs)) if abs(outputs[i] - labels[i]) < 0.1])
            total_count += labels.size(0)
            
        # Print loss and accuracy for each epoch
        print(f'Epoch {epoch+1}: Loss = {total_loss/total_count}, Accuracy = {total_correct/total_count}')
        progress.update(task, advance=1)

Output()

/home/loong/miniconda3/envs/PyTorch/lib/python3.10/site-packages/torch/nn/functional.py:5527: UserWarning: 1Torch 
was not compiled with memory efficient attention. (Triggered internally at 
../aten/src/ATen/native/transformers/hip/sdp_utils.cpp:505.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)

Epoch 1: Loss = 0.005962874307825522, Accuracy = 0.22516166960611406

Epoch 2: Loss = 0.0024467922981471473, Accuracy = 0.2727807172251617

Epoch 3: Loss = 0.002086218052282535, Accuracy = 0.3115814226925338

...

Epoch 46: Loss = 0.0008531196567414439, Accuracy = 0.5202821869488536

Epoch 47: Loss = 0.0008002022488797188, Accuracy = 0.5261610817166372

Epoch 48: Loss = 0.0008045516657407041, Accuracy = 0.5220458553791887

Epoch 49: Loss = 0.0007832474568021045, Accuracy = 0.527336860670194

Epoch 50: Loss = 0.0007382743241268701, Accuracy = 0.5349794238683128

In [8]:
# Save model
torch.save(model.state_dict(), 'transformer.ckpt')

In [10]:
# Inference
model.load_state_dict(torch.load('transformer.ckpt'))
model = model.to(device)
model.eval()

TransformerModel(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=21, out_features=21, bias=True)
          )
          (linear1): Linear(in_features=21, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=21, bias=True)
          (norm1): LayerNorm((21,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((21,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((21,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
  

In [13]:
correct = 0
total = 0

with torch.no_grad():
    for features_1, features_2, labels in test_loader:
        # Remove last day's features as we don't have the next day's mood value
        src = features_1.float().to(device)
        tgt = features_2.float().to(device)
        labels = labels.float().squeeze().to(device)

        outputs = model(src, tgt).squeeze()

        # _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # correct += (predicted == labels).sum().item()
        # correct += (outputs == labels).sum().item()
        # outputs和labels为list，若outputs和labels中的一项的差值小于0.1，则认为这一项预测正确
        correct += sum([1 for i in range(len(outputs)) if abs(outputs[i] - labels[i]) < 0.1])

In [14]:
print('Accuracy: %d %%' % (100 * correct / total))

Accuracy: 67 %
